In [1]:
from flask import *
import pandas as pd
app = Flask(__name__)
import json
import urllib3
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from forex_python.converter import CurrencyRates

In [5]:
c = CurrencyRates()
c.get_rate('USD', 'INR')

63.517

In [13]:
# Get Todays date
i = datetime.now()
now = i.strftime('%Y-%m-%d ')

data = pd.read_excel('/Users/smahimkar/Desktop/invest/investment/invest_tables/data.xlsx')
#data.set_index(['Name'], inplace=True)
#data.index.name=None
cryptocurrency_temp = data.loc[data.Type == 'Cryptocurrency']
## Get todays Value of cryptocurrencies
http = urllib3.PoolManager()
r = http.request('GET', 'https://api.coinmarketcap.com/v1/ticker/')
all_data = json.loads(r.data.decode('utf-8'))
all_data_df = pd.DataFrame(all_data)
cryptocurrency_popular = cryptocurrency_temp.merge(all_data_df[['id','price_usd']],left_on='Name',right_on='id',how='inner')
cryptocurrency_popular.Units = cryptocurrency_popular.Units.apply(float)
cryptocurrency_popular.price_usd = cryptocurrency_popular.price_usd.apply(float)
cryptocurrency_popular.Todays_Value = cryptocurrency_popular.price_usd * cryptocurrency_popular.Units
cryptocurrency_popular.drop(['id','price_usd'],axis=1,inplace=True)
cryptocurrency_popular.Maturity_Date = now

cryptocurrency_ico = cryptocurrency_temp[~cryptocurrency_temp.Name.isin(cryptocurrency_popular.Name.unique())]
cryptocurrency_ico.Maturity_Date = 'TBD'
cryptocurrency_final = pd.concat([cryptocurrency_popular,cryptocurrency_ico])

deposit_temp = data.loc[data.Type == 'Deposit']
deposit_temp.Currency = 'USD'
deposit_temp.Money_Invested = deposit_temp.Money_Invested / c.get_rate('USD', 'INR')
deposit_temp.Todays_Value = deposit_temp.Todays_Value /  c.get_rate('USD', 'INR')

new_data = pd.concat([cryptocurrency_final,deposit_temp])
# new_data.set_index(['Name'], inplace=True)
# new_data.index.name=None
# cryptocurrency = new_data.loc[new_data.Type == 'Cryptocurrency']
# deposit = new_data.loc[new_data.Type == 'Deposit']

/Users/smahimkar/Desktop/invest/investment/flask_env/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/smahimkar/Desktop/invest/investment/flask_env/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [14]:
deposit

,Units,Currency,Money_Invested,Todays_Value,Date_of_Investment,Maturity_Date,Link,Type
icici_rd,NaN,USD,4723.144985,4902.404081,2017-07-03,2018-07-03 00:00:00,https://www.icicibank.com,Deposit


In [18]:
total = new_data.groupby('Currency').agg({'Money_Invested':'sum','Todays_Value':'sum'}).reset_index()
total

,Currency,Todays_Value,Money_Invested
0,USD,6204.499658,5831.744985


In [11]:
new_data.Money_Invested.sum()
new_data.Money_Invested.sum()

5831.744984807217